# Проект Анализ сервиса вопросов и ответов по программированию

**Цель проекта:**  
С помощью SQL посчитать и визуализировать ключевые метрики сервис-системы вопросов и ответов о программировании.

**Описание проекта:**  
Написаны все сложные SQL-запросы для подсчёта требуемых значений и метрик.

# Часть 1

## Задание 1
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id = (
            SELECT id
            FROM stackoverflow.post_types
            WHERE type = 'Question'
            )
AND (favorites_count >= 100
OR score > 300)

## Задание 2
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH a AS(
SELECT DATE_TRUNC('day', creation_date)::date AS date,
       COUNT(id) AS q_count
FROM stackoverflow.posts
WHERE post_type_id = (
            SELECT id
            FROM stackoverflow.post_types
            WHERE type = 'Question'
            )
AND creation_date BETWEEN '2008-11-01' AND '2008-11-19' 
GROUP BY DATE_TRUNC('day', creation_date)::date
)

SELECT ROUND(SUM(q_count) / COUNT(date))
FROM a

## Задание 3
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE DATE_TRUNC('day', u.creation_date)::date = DATE_TRUNC('day', b.creation_date)::date

## Задание 4
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT post_id)
FROM stackoverflow.votes
WHERE post_id IN(
SELECT DISTINCT id
FROM stackoverflow.posts
WHERE user_id = (
SELECT id
FROM stackoverflow.users
WHERE display_name = 'Joel Coehoorn'
)
)

## Задание 5
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
      RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id

## Задание 6
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id)
FROM stackoverflow.votes
WHERE vote_type_id =
(SELECT id
FROM stackoverflow.vote_types
WHERE name = 'Close')
GROUP BY user_id
ORDER BY COUNT(id) DESC, user_id DESC
LIMIT 10

## Задание 7
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id),
       DENSE_RANK() OVER (ORDER BY COUNT(id) DESC)
FROM stackoverflow.badges
WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY COUNT(id) DESC, user_id
LIMIT 10

## Задание 8
Сколько в среднем очков получает пост каждого пользователя?  
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg_score
FROM stackoverflow.posts
WHERE score != 0
  AND title IS NOT NULL 

## Задание 9
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN(
SELECT user_id
FROM stackoverflow.badges
GROUP BY user_id
HAVING COUNT(id) > 1000
)
AND title IS NOT NULL

## Задание 10
Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). 
Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. 

Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
WITH a AS(
       SELECT *,
       CASE
       WHEN views >= 350 THEN 1
       WHEN views <350 AND views >=100 THEN 2
       ELSE 3
       END AS group_t
FROM stackoverflow.users
WHERE location LIKE '%Canada%'
)

SELECT id,
       views,
       group_t
FROM a 
WHERE views >0

## Задание 11
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH a AS(
       SELECT id, views,
       CASE
       WHEN views >= 350 THEN 1
       WHEN views <350 AND views >=100 THEN 2
       ELSE 3
       END AS group_t
FROM stackoverflow.users
WHERE location LIKE '%Canada%'
  AND views != 0  
),

b AS(
SELECT *,
       MAX(views) OVER (PARTITION BY group_t) AS max_views
FROM a 
)

SELECT id, group_t, views
FROM b
WHERE views = max_views
ORDER BY views DESC, id

## Задание 12
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. 
Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH a AS(
SELECT EXTRACT(DAY FROM creation_date) AS day,
       COUNT(id) AS new_users
FROM stackoverflow.users
WHERE DATE_TRUNC('month', creation_date)::date = '2008-11-01'
GROUP BY EXTRACT(DAY FROM creation_date)
)

SELECT *,
       SUM(new_users) OVER (ORDER BY day)
FROM a

## Задание 13
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 
Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT u.id,
       MIN(p.creation_date) OVER(PARTITION BY u.id) - u.creation_date
FROM stackoverflow.users AS u 
JOIN stackoverflow.posts AS p ON u.id=p.user_id

# Часть 1

## Задание 1
Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DISTINCT DATE_TRUNC('month', creation_date)::date AS month,
       SUM(views_count) OVER(PARTITION BY DATE_TRUNC('month', creation_date)) AS sum_views
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date)=2008
ORDER BY sum_views DESC

## Задание 2
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
  AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
  AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(*) > 100
ORDER BY display_name;

## Задание 3
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date AS month,
       COUNT(id)
FROM stackoverflow.posts
WHERE user_id IN(
SELECT user_id
FROM stackoverflow.posts
WHERE DATE_TRUNC('month', creation_date)::date = '2008-12-01'
  AND user_id IN(
  SELECT id
FROM stackoverflow.users
WHERE DATE_TRUNC('month', creation_date)::date = '2008-09-01'
)
)
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY DATE_TRUNC('month', creation_date)::date DESC

## Задание 4
Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER(PARTITION BY user_id ORDER BY user_id, creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date

## Задание 5
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH a AS(
SELECT user_id,
       COUNT(DISTINCT DATE_TRUNC('day', creation_date)::date) AS active_days
FROM stackoverflow.posts
WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY user_id
)

SELECT ROUND(AVG(active_days))
FROM a

## Задание 6
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? 

Отобразите таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

In [ ]:
WITH a AS(
SELECT EXTRACT(MONTH FROM creation_date) AS month,
       COUNT(id) AS posts_per_month,
       LAG(COUNT(id)) OVER(ORDER BY EXTRACT(MONTH FROM creation_date)) posts_last_month
FROM stackoverflow.posts
WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY EXTRACT(MONTH FROM creation_date)
)

SELECT month, 
       posts_per_month,
       ROUND((posts_per_month/posts_last_month::numeric*100)-100, 2) AS diff_perc
FROM a 

## Задание 7
Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. 

Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT DISTINCT EXTRACT(WEEK FROM creation_date) AS week,
       MAX(creation_date) OVER(ORDER BY EXTRACT(WEEK FROM creation_date))
FROM stackoverflow.posts
WHERE user_id =(
SELECT user_id
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY COUNT(id) DESC
LIMIT 1
)
AND DATE_TRUNC('month', creation_date) = '2008-10-01'